In [29]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import Hour, Minute
from pandas.tseries.offsets import Day, MonthEnd
from pandas.tseries.offsets import Hour
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as mse
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from tensorflow.keras.utils import set_random_seed as set_seed
set_seed(777)


In [30]:


#df = pd.read_csv('Wind_deseason.csv')
df = pd.read_csv('Wind_deseason_15.csv', parse_dates=[0], index_col=0)


In [31]:
def TrainTest(df):
    
    num_rows = df.shape[0]
    
    num_training = (num_rows * 8)//10
   
    #train = df.iloc[0:num_training]
    #test = df.iloc[num_training:num_rows]
    
    
    return num_training 
        
            

In [32]:
def lags(df, n):
    
    df_new = df
    
    for i in range(n):
        name = "lag "+ str(i+1)
        df_new[name] = df['309_WIND_1'].shift(i+1)
    
    return df_new

In [33]:

def model1Train(X,Y):
     
    mod = Sequential()
    mod.add(LSTM(6, activation='sigmoid', input_shape=(107, 1), return_sequences = True))
    mod.add(LSTM(6, activation='sigmoid', input_shape=(107, 1), return_sequences = True))
    mod.add(LSTM(6, activation='sigmoid', input_shape=(107, 1)))
    mod.add(Dense(1))
    mod.compile(optimizer='adam', loss='mse', metrics = ['accuracy'])
       
        
    print(mod.summary())
        
    mod.fit(x = X, y = Y,epochs=20,verbose = 1)#keep low for now, edit later
    
    return mod
    
    
    

In [34]:
def model2Train(gen):
     
    mod = Sequential()
    mod.add(LSTM(1, activation='sigmoid', input_shape=(107, 1)))
   
    mod.add(Dense(1))
    mod.compile(optimizer='adam', loss='mse', metrics = ['accuracy'])
       
        
    print(mod.summary())
        
    mod.fit(gen,epochs=3,verbose = 1)#keep low for now, edit later
    
    return mod
    

In [35]:
def TestModel(X,test, mod, scaler):
    
    MSEs = []
    
    for i in range(len(X)):
        array = np.array(X[i])
        
        pred = mod.predict(array.reshape(1,107,1))
        print(pred)
        #pred = scaler.inverse_transform(pred)
        
        unscaled_pred = scaler.inverse_transform(pred)
        unscaled_test = scaler.inverse_transform([[test[i]]])
       
        
        MSEs.append(mse(unscaled_test[0],unscaled_pred[0]))
        
        
    return MSEs

In [36]:
#def modelNaiveTest(df):

    
    
    


In [37]:
def runModels(df, last):
    
    scaler = MinMaxScaler()
    
    bound = TrainTest(df)
    
    train = df[0:bound]
    test = df[bound:]
    
    
    scaler.fit(train['309_WIND_1'].to_frame())
    
    scaled_df = scaler.transform(df['309_WIND_1'].to_frame())
    scaled_last = scaler.transform(last['309_WIND_1'].to_frame())
    scaled_df = np.append(scaled_last, scaled_df)
    
    
    generator = TimeseriesGenerator(scaled_df[107:bound+107], scaled_df[107:bound+107], length=107, batch_size=1)
                                      
    

    scaled_df = pd.DataFrame(scaled_df, columns = ['309_WIND_1'])
    
    lag = lags(scaled_df, 107)
    
    lag = lag[lag.columns[::-1]]
    
    lagged_train = lag.iloc[107:bound+107]
    lagged_test = lag.iloc[bound+107:]

    
    
    X_test = lagged_test.drop('309_WIND_1', axis = 1).values.tolist()
    Y_test = lagged_test['309_WIND_1'].values.tolist()
  
    
    model2 = model2Train(generator)
    
    Tests = TestModel(X_test, Y_test, model2, scaler)
    
    
    print(Tests)
    
   
    
    return Tests#, mse2

    

In [38]:


df1 = df[df.index.month.isin([1,2,3])]
df1 = df1.iloc[:-1 , :]
print(len(df1))

df2 = df[df.index.month.isin([4,5,6])]

df3 = df[df.index.month.isin([7,8,9])]

df4 = df[df.index.month.isin([10,11,12])]

In [39]:
# mse1 = runModels(df1,df4.iloc[-107:])


In [40]:
#mse3 = runModels(df2,df1.iloc[-107:])

In [41]:
mse5 = runModels(df3, df2.iloc[-107:])

In [42]:
#mse7 = runModels(df4, df3.iloc[-107:])

In [43]:
#print('season 1')

#print('model 1:')
#print(mse1)
#print('model Naive:')
#print(mse2)


In [44]:
#print('season 1')

#print('model 1:')
#print(mse3)
#print('model Naive:')
#print(mse4)


In [45]:
#print('season 1')

#print('model 1:')
#print(mse5)
#print('model Naive:')
#print(mse6)


In [46]:
#print('season 1')

#print('model 1:')
#print(mse7)
#print('model Naive:')
#print(mse8)


In [47]:
#'Season 1':mse2,'Season 2':mse4,'Season 4':mse8
d = {'Season 3':mse5}
#df_errors_Naive = pd.DataFrame({k:pd.Series(v) for k,v in d.items()})
#d = {'Season1':mse1,'Season2':mse3,'Season3':mse5,'Season4':mse7}
df_errors_LSTM = pd.DataFrame({k:pd.Series(v) for k,v in d.items()})
d#f_errors_Naive.to_csv('Naive_errors.csv')
df_errors_LSTM.to_csv('LSTM_errors_not_scaled_s3.csv')

